In [29]:
import pandas as pd
import logging
import swc_simple_client as swc


In [25]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    filename='shark_notebook.log',
    level=logging.INFO,
)

In [26]:
base_url = "https://fantasyfootbal-api-service.onrender.com"

In [31]:
week_api_response = swc.call_api_endpoint(base_url, "/v0/teams/")#swc.LIST_TEAMS_ENDPOINT)
weeks_df = pd.DataFrame(week_api_response.json())
weeks_df['year'] = weeks_df['week_number'].str.slice(0,4).astype(int)
weeks_df['week'] = weeks_df['week_number'].str.slice(4,6).astype(int)

weeks_df = weeks_df.query('week <= 14')
max_totals_grouped_df = weeks_df.groupby('year').agg(
    ppr_12_max_points = ('ppr_12_max_points', 'sum'),
    half_ppr_8_max_points=('half_ppr_8_max_points', 'sum')
)

display(max_totals_grouped_df)

KeyError: 'week_number'

In [34]:
league_api_response = swc.call_api_endpoint(base_url,swc.LIST_LEAGUES_ENDPOINT)
leagues_df = pd.DataFrame(league_api_response.json())
leagues_df = leagues_df.drop(columns=['teams','last_changed_date'])
display(leagues_df)

,league_id,league_name,scoring_type
0,5001,Pigskin Prodigal Fantasy League,PPR
1,5002,Recurring Champions League,Half-PPR
2,5003,AHAHFZZFFFL,Standard
3,5004,Gridiron Gurus Fantasy League,PPR
4,5005,Best League Ever,PPR


In [39]:
team_api_response = swc.call_api_endpoint(base_url,"/v0/teams/")

teams_parsed_df = pd.json_normalize(team_api_response.json(), 
                                    ['league_id', 'team_id', 'team_name'])

teams_parsed_df['year'] = (teams_parsed_df['week_number']
                           .str.slice(0, 4).astype(int))
teams_parsed_df['week'] = (teams_parsed_df['week_number']
                           .str.slice(4, 6).astype(int))

#get only regular season teams
teams_regular_season_df = teams_parsed_df.query('week <= 14')

#get team season totals
team_totals_df = teams_regular_season_df.groupby(
    ['league_id', 'team_id', 'year'], as_index = False
    )['fantasy_points'].sum()

team_totals_df.head()

TypeError: 'int' object is not iterable

In [40]:
league_stats_df = team_totals_df.groupby(['league_id','year']).agg(
    league_points_sum=('fantasy_points', 'sum'),
    league_points_mean=('fantasy_points', 'mean'),
    league_points_stdev=('fantasy_points', 'std'),
    league_balance_score=('fantasy_points', 
                          lambda x: (100 -(x.std() / x.mean()) * 100))
).reset_index()

display(league_stats_df.sort_values(by='league_balance_score', ascending=False))

NameError: name 'team_totals_df' is not defined

In [41]:
league_stats_with_league_max_df = (league_stats_df[
    ['league_id','year', 'league_points_sum','league_balance_score']]
               .merge(max_totals_grouped_df,left_on = 'year', right_on='year'))

combined_metrics_df = (leagues_df[
    ['league_id','league_name','scoring_type', 'league_size']]
    .merge(league_stats_with_league_max_df, 
           left_on = 'league_id', right_on = 'league_id'))

combined_metrics_df['league_juice_score'] = combined_metrics_df.apply(
    lambda row: (
        100 * (row['league_points_sum'] / row['ppr_12_max_points'])
        if (row['scoring_type'] == 'PPR' and row['league_size'] == 12) 
        else (
            100 * (row['league_points_sum'] / row['half_ppr_8_max_points'])
            if (row['scoring_type'] == 'Half-PPR' and row['league_size'] == 8) 
            else None
        )
    ),
    axis=1
)

combined_metrics_df = (combined_metrics_df.drop(
    columns=['scoring_type','league_size','league_points_sum'
             ,'ppr_12_max_points','half_ppr_8_max_points',] )
)
display(combined_metrics_df)

NameError: name 'league_stats_df' is not defined

In [42]:
combined_metrics_df['shark_league_score'] = combined_metrics_df.apply(
    lambda league: (2 * league['league_juice_score']) +  league['league_balance_score'],
    axis=1
)
display(combined_metrics_df.sort_values(by='shark_league_score', ascending=False))

NameError: name 'combined_metrics_df' is not defined